In [ ]:
import os
import random
from glob import glob
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from model import Transformer
from utils import VectorizeChar, DisplayOutputs, CustomSchedule, path_to_features , wer, cer
import pandas as pd

In [ ]:
def set_seeds(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)

set_seeds()

In [ ]:
# to tokenize the text
vectorizer = VectorizeChar(100)

In [ ]:
max_target_len = 100  # all transcripts in out data are < 100 characters
print("vocab size", len(vectorizer.get_vocabulary()))

vocab size 95


In [ ]:
#  utilities for data preparation for training and testing [Convert to TF Dataset]
def encode(path,txt):
  """Encode a text into a sequence of vectors."""
  txt = txt.numpy().decode('utf8')
  y = tf.convert_to_tensor(vectorizer(txt),dtype=tf.int64)
  x = path_to_features(path)
  return x,y
def tf_encode(path,txt):
  """ util py function to be used with tensors."""
  x,y = tf.py_function(encode, [path,txt], [tf.float32,tf.int64])   
  return x,y
def create_tf_dataset(data, batch_size=4):
  """Create a tf.data.Dataset from the given data."""
  dataset = tf.data.Dataset.from_tensor_slices((np.array(data["filename"].values),np.array(data["transcript"].values)))
  dataset = dataset.map(tf_encode, num_parallel_calls=tf.data.experimental.AUTOTUNE)
  dataset = dataset.padded_batch(batch_size, padded_shapes=([None,20], [None]))
  dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
  dataset = dataset.map(lambda x, y: {"source": x, "target": y})
  return dataset

## Create & train the end-to-end model

In [ ]:

df = pd.read_excel('data/iam.xlsx')
df['filename'] = df['file_path'].apply(lambda x: "data/content/bin_files/"+x.split('/')[-1].replace('.xml','.bin'))
# split data into train and test 
msk = np.random.rand(len(df)) < 0.95
train_data = df[msk]
dev_data = df[~msk]
# create tf dataset for training and validation
ds = create_tf_dataset(train_data, batch_size=32)
val_ds = create_tf_dataset(dev_data, batch_size=8)

In [ ]:
len(train_data),len(dev_data)

(11524, 663)

In [ ]:
# to display the output of the model after each epoch for the first batch of the validation set
batch = next(iter(val_ds))
# The vocabulary to convert predicted indices into characters
idx_to_char = vectorizer.get_vocabulary()
display_cb = DisplayOutputs(
    batch, idx_to_char, target_start_token_idx=2, target_end_token_idx=3
)  # set the arguments as per vocabulary index for '<' and '>'

# create the model and compile it
model = Transformer(
    num_hid=100,
    num_head=2,
    num_feed_forward=256,
    target_maxlen=max_target_len,
    num_layers_enc=4,
    num_layers_dec=1,
    num_classes=95,
)
loss_fn = tf.keras.losses.CategoricalCrossentropy(
    from_logits=True, label_smoothing=0.1,
)

learning_rate = CustomSchedule(
    init_lr=0.00001,
    lr_after_warmup=0.001,
    final_lr=0.00001,
    warmup_epochs=15,
    decay_epochs=85,
    steps_per_epoch=len(ds),
)
optimizer = keras.optimizers.Adam(learning_rate)
# optimizer = keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss=loss_fn)


In [ ]:
#  train the model
history = model.fit(ds, validation_data=val_ds, callbacks=[display_cb], epochs=20)

Epoch 1/20
360/361 [============================>.] - ETA: 0s - loss: 2.2369target:     <except by roasting and boiling. When>
prediction: <the an the the the the the the an the and>

target:     <Achilles gave a Royal feast the principal dish>
prediction: <the an the the the the the the an the and>

target:     <aversion . Shelley , for example , had a>
prediction: <the an the the the the the the an the and>

target:     <Dante Gabriel Rossetti>
prediction: <the an the the the the the the an the and>

target:     <the marks on the back>
prediction: <the an the the the the the the an the and>

target:     <is so languorous , so passionate ,>
prediction: <the an the the the the the the an the>

target:     <foods for which they have>
prediction: <the an the the the the the the an the and>

target:     <"one man's meat is another man's poison",>
prediction: <the an the the the the the the an the and>

361/361 [==============================] - 71s 196ms/step - loss: 2.2380 - val_loss: 2.

In [ ]:
history = model.fit(ds, validation_data=val_ds, callbacks=[display_cb], epochs=20)

Epoch 1/20
360/361 [============================>.] - ETA: 0s - loss: 0.8428target:     <except by roasting and boiling. When>
prediction: <orcept ly roasting and boiling. When>

target:     <Achilles gave a Royal feast the principal dish>
prediction: <Helilles gave a loyal feast the principatis>

target:     <aversion . Shelley , for example , had a>
prediction: <aversion. Shelles, for explest no >

target:     <Dante Gabriel Rossetti>
prediction: <I ante Cahbiel Rosssesstelt>

target:     <the marks on the back>
prediction: <the marks on the back>

target:     <is so languorous , so passionate ,>
prediction: <is so langurous, so pestionstente, and>

target:     <foods for which they have>
prediction: <toods for which they have>

target:     <"one man's meat is another man's poison",>
prediction: <"Ye naris neat is unother man' s yoin >

361/361 [==============================] - 70s 195ms/step - loss: 0.8428 - val_loss: 1.0059
Epoch 2/20
360/361 [============================>.] - ETA

In [ ]:
history = model.fit(ds, validation_data=val_ds, callbacks=[display_cb], epochs=20)

Epoch 1/20
360/361 [============================>.] - ETA: 0s - loss: 0.6664target:     <except by roasting and boiling. When>
prediction: <except by roasting and boiling. When>

target:     <Achilles gave a Royal feast the principal dish>
prediction: <Achill is gave a Royal feast the principal >

target:     <aversion . Shelley , for example , had a>
prediction: <averesion. Shelley, four eamples ha>

target:     <Dante Gabriel Rossetti>
prediction: <Danto Govel Siel Rossessed itit>

target:     <the marks on the back>
prediction: <the marks on the back>

target:     <is so languorous , so passionate ,>
prediction: <is so so knoway, so po passionts and>

target:     <foods for which they have>
prediction: <toods for which they have>

target:     <"one man's meat is another man's poison",>
prediction: <"one mak is neat is unother unaton' >

361/361 [==============================] - 75s 208ms/step - loss: 0.6668 - val_loss: 0.8883
Epoch 2/20
360/361 [============================>.] - ET

In [ ]:
history = model.fit(ds, validation_data=val_ds, callbacks=[display_cb], epochs=20)

Epoch 1/20
360/361 [============================>.] - ETA: 0s - loss: 0.5970target:     <except by roasting and boiling. When>
prediction: <except ty roasting and boiling. When>

target:     <Achilles gave a Royal feast the principal dish>
prediction: <Helilles gavea Royal feast the principal dish>

target:     <aversion . Shelley , for example , had a>
prediction: <awersion. Shelley , for eamples had a>

target:     <Dante Gabriel Rossetti>
prediction: <Dante Gahriel Rossesseltutitutit>

target:     <the marks on the back>
prediction: <the marks on the back>

target:     <is so languorous , so passionate ,>
prediction: <is so kangurous o po passionate, and>

target:     <foods for which they have>
prediction: <toods for which they have>

target:     <"one man's meat is another man's poison",>
prediction: <"re man's neat is unnuther is said. " oid>

361/361 [==============================] - 70s 193ms/step - loss: 0.5973 - val_loss: 0.8909
Epoch 2/20
360/361 [==========================

## Evalute the model

In [ ]:
test_dataset = create_tf_dataset(dev_data, batch_size=32)
idx_to_char = vectorizer.get_vocabulary()

result=[]
for batch_idx, batch in enumerate( test_dataset):
  print(batch_idx)
  source = batch["source"]
  target = batch["target"].numpy()
  bs = tf.shape(source)[0]
  preds = model.generate(source, 2)
  preds = preds.numpy()
  for i in range(bs):
      target_text = "".join([idx_to_char[_] for _ in target[i, :]])
      prediction = ""
      for idx in preds[i, :]:
          prediction += idx_to_char[idx]
          if idx == 3:
              break
      result.append({"target":target_text.replace('-',''),"prediction":prediction})
      # print(f"target:     {target_text.replace('-','')}")
      # print(f"prediction: {prediction}\n")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [ ]:
# Convert the result to a pandas dataframe 
prediction_result = pd.DataFrame(result)
prediction_result.head()

,target,prediction
0,<except by roasting and boiling. When>,<except by roasting and boiling. When>
1,<Achilles gave a Royal feast the principal dish>,<Achilles gave a Royal feast the prinepal dishn.>
2,"<aversion . Shelley , for example , had a>",<auresion. Shellency for eamples had a>
3,<Dante Gabriel Rossetti>,<Danto Gabriel Rosse isselt>
4,<the marks on the back>,<the marks on the back>


In [ ]:
#  Normalize the results 
prediction_result['prediction'] = prediction_result['prediction'].apply(lambda x:x.replace('<','').replace('>',''))
prediction_result['target'] = prediction_result['target'].apply(lambda x:x.replace('<','').replace('>',''))

In [ ]:
#  Calculate the accuracy of the model using character error rate (CER) and word error rate (WER)
prediction_result['wer'] = prediction_result.apply(lambda x: wer(x.target, x.prediction) , axis=1)
prediction_result['cer'] = prediction_result.apply(lambda x: cer(x.target, x.prediction) , axis=1)
print(f" cer {prediction_result['cer'].mean():.5f} wer {prediction_result['wer'].mean():.5f}")

In [ ]:
#  Write the results to a excel file
prediction_result.to_excel('Results/iam_results_01.xlsx')

In [ ]:
#  Save the model 
model.save_weights('Models/iam80.h5')